In [ ]:
import sys
sys.path.append('../../')

In [1]:
import pandas as pd

In [ ]:
dms = pd.read_csv('./DMS/DMS_S_annotated_with_counts.csv',index_col = 0)
dms = dms[(dms.label.str.contains('mask') == False) & (dms.label.str[0] != dms.label.str[-1])].drop_duplicates()

In [ ]:
single_nucleotide_spike = pd.read_csv('./Benchmarking/Evescape/spike_dist_one_scores_gisaid.csv')

In [133]:
def variant_counts (scores,voc_mutations):

  score_counts = []
  proportions = [[0,0.1],[0.1,0.25],[0.25,1]]
  for proportion in proportions:
    for score in ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",'relative_grammaticality_with_eve',
                  'Evescape Fitness','Evescape','IND Mutability Score','DCA Mutability Score','Entry','Binding','Escape','RBD Wuhan-Hu-1 Binding',
                  'RBD Variant Average Binding','RBD Wuhan-Hu-1 Expression','RBD Variant Average Expression']:
        
        
        #Sort DMS on score
        sorted_dms = scores.sort_values(score,ascending=False)
        subset_length = len(sorted_dms.dropna(subset=score))
        sorted_dms = sorted_dms[int(subset_length*proportion[0]):int(subset_length*proportion[1])]
        #Full Spike Row
        full_spike = pd.merge(sorted_dms,voc_mutations,left_on='label',right_on='mutation',how='left')[["B.1.1.7","B.1.351","B.1.617.2","P.1","BA.1","BA.2","BA.4","BA.2.12.1","BA.2.75"]]
        row = pd.DataFrame(full_spike.sum()).T
        row['feature'] = score
        row['region'] = 'spike'
        row['proportion'] = proportion[1]
        score_counts.append(row)

        
        #Sort DMS on score
        sorted_rbd_dms = scores.sort_values(score,ascending=False)
        sorted_rbd_dms = sorted_rbd_dms[(sorted_rbd_dms.pos >= 319) & (sorted_rbd_dms.pos <= 541)]
        rbd_subset_length = len(sorted_rbd_dms.dropna(subset=score))
        sorted_rbd_dms = sorted_rbd_dms[int(rbd_subset_length*proportion[0]):int(rbd_subset_length*proportion[1])]
        #RBD Row
        rbd = pd.merge(sorted_rbd_dms,voc_mutations,left_on='label',right_on='mutation',how='left')[["B.1.1.7","B.1.351","B.1.617.2","P.1","BA.1","BA.2","BA.4","BA.2.12.1","BA.2.75"]]
        row = pd.DataFrame(rbd.sum()).T
        row['feature'] = score
        row['region'] = 'rbd'
        row['proportion'] = proportion[1]
        score_counts.append(row)
  score_counts = pd.concat(score_counts) 
  score_counts = score_counts.melt(id_vars=['feature','region','proportion'],var_name='voc',value_name='count')
  return score_counts

In [137]:
variant_counts(dms,single_nucleotide_spike).to_csv('voc_indentification_updated.csv',index=False)

# No Nucleotide Restrictions 

In [ ]:
voc_seqs = pd.read_csv('./Benchmarking/Evescape/Initial_Vocs_Eve.csv',index_col=0)[['lineage','mutations']]
voc_seqs

,lineage,mutations
32,BA.1,"A67V, H69del, V70del, T95I, G142del, V143del, ..."
100,BA.2.75,"T19I, G142D, K147E, W152R, F157L, I210V, V213G..."
178,BA.2.12.1,"T19I, L24del, P25del, P26del, A27S, G142D, V21..."
308,BA.2,"T19I, L24del, P25del, P26del, A27S, G142D, V21..."
605,BA.4,"T19I, L24del, P25del, P26del, A27S, H69del, V7..."
743,B.1.1.7,"H69del, V70del, Y144del, N501Y, A570D, D614G, ..."
801,B.1.351,"L18F, D80A, D215G, K417N, E484K, N501Y, D614G,..."
806,P.1,"L18F, T20N, P26S, D138Y, R190S, K417T, E484K, ..."
843,B.1.617.2,"T19R, E156del, F157del, R158G, L452R, T478K, D..."


In [5]:
mutations = []
for i,row in voc_seqs.iterrows():
    for mutation in row['mutations'].split(', '):
        if 'del' in mutation:
            continue
        mutations.append({'lineage':row['lineage'],'mutation':mutation})
mutations = pd.DataFrame(mutations)
mutations = mutations.join(mutations.lineage.str.get_dummies())
mutations.drop('lineage',axis=1,inplace=True)
mutations = mutations.groupby('mutation').sum().reset_index()
mutations['is_voc'] = 1
mutations.sort_values('mutation')


,mutation,B.1.1.7,B.1.351,B.1.617.2,BA.1,BA.2,BA.2.12.1,BA.2.75,BA.4,P.1,is_voc
0,A27S,0,0,0,0,1,1,0,1,0,1
1,A570D,1,0,0,0,0,0,0,0,0,1
2,A67V,0,0,0,1,0,0,0,0,0,1
3,A701V,0,1,0,0,0,0,0,0,0,1
4,A706V,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
66,V1176F,0,0,0,0,0,0,0,0,1,1
67,V213G,0,0,0,0,1,1,1,1,0,1
68,W152R,0,0,0,0,0,0,1,0,0,1
69,Y145D,0,0,0,1,0,0,0,0,0,1


In [6]:
mutations.sum()

mutation     A27SA570DA67VA701VA706VD1118HD138YD215GD405ND6...
B.1.1.7                                                      7
B.1.351                                                      9
B.1.617.2                                                    8
BA.1                                                        30
BA.2                                                        28
BA.2.12.1                                                   30
BA.2.75                                                     33
BA.4                                                        29
P.1                                                         12
is_voc                                                      71
dtype: object

In [136]:
variant_counts(dms,mutations).to_csv('voc_indentification_updated_all.csv',index=False)

# Monthly Mutations 

In [ ]:
mut_counts_per_month = pd.read_csv('./Benchmarking/Evescape/single_mutant_count_by_month.csv')

#Convert to cumulative counts
mut_counts_per_month[mut_counts_per_month.columns[1:]] = mut_counts_per_month[mut_counts_per_month.columns[1:]].cumsum(axis=1)

#reshape
mut_counts_per_month = pd.melt(mut_counts_per_month,
                 id_vars=["mutation"],
                 var_name="sample_date",
                 value_name="counts")

#subset to counts>100
mut_counts_per_month = mut_counts_per_month[mut_counts_per_month.counts > 100]
mut_counts_per_month = mut_counts_per_month.sort_values(by="sample_date")
mut_counts_per_month = mut_counts_per_month.drop_duplicates(subset="mutation")
mut_counts_per_month["first_seen"] = mut_counts_per_month.sample_date

mut_counts_per_month['RBD'] = (mut_counts_per_month['mutation'].str[1:-1].astype(int) >=319) & (mut_counts_per_month['mutation'].str[1:-1].astype(int) <=541)  
mut_counts_per_month.rename(columns={'counts':'count'},inplace=True)
mut_counts_per_month.sort_values(['sample_date','mutation'])

,mutation,sample_date,count,first_seen,RBD
8492,D614G,2020-03,1348,2020-03,False
10820,P1263L,2020-04,110,2020-04,False
15768,D839Y,2020-05,158,2020-05,False
15305,D936Y,2020-05,305,2020-05,False
14579,L54F,2020-05,110,2020-05,False
...,...,...,...,...,...
111294,V656M,2022-08,103,2022-08,False
111533,V90R,2022-08,101,2022-08,False
110867,Y313I,2022-08,101,2022-08,False
110564,Y369F,2022-08,121,2022-08,True


In [8]:
def muts_per_month(scores, feature_methods, monthly_counts, proportions):
    all_values = []
    for region in ['spike','rbd']:
        if region == 'rbd':
            region_scores = scores[(scores.pos >= 319) & (scores.pos <= 541)]
            region_monthly = monthly_counts[monthly_counts.RBD]
        else:
            region_scores = scores
            region_monthly = monthly_counts[monthly_counts.RBD]
        for proportion in proportions:
            for value in feature_methods:
                #Top Proportion
                sorted_scores = region_scores.sort_values(value[0],ascending=False)
                sorted_scores = sorted_scores.dropna(subset=value[0])
                subset_length = len(sorted_scores)
                sorted_scores = sorted_scores[:int(subset_length*proportion)]

                #Subset in data
                monthly_mutations =region_monthly.merge(sorted_scores,left_on='mutation',right_on='label',how='left') 
                monthly_mutations=monthly_mutations[monthly_mutations.label.notnull()]
                for date in monthly_counts.sample_date.unique():
                    subset = monthly_mutations[monthly_mutations.sample_date <= date]
                    row = pd.DataFrame([date,len(subset),len(subset)/len(sorted_scores),proportion,value[0],value[1],region],index = ['date','count','percentage','proportion','feature','method','region']).T
                    all_values.append(row)
    return pd.concat(all_values)

In [9]:
datasets = [

    ['Entry','Experimental Full Spike'],
    ['Binding','Experimental Full Spike'],
    ['Escape','Experimental Full Spike'],
    
    ['RBD Wuhan-Hu-1 Binding','Experimental'],
    ['RBD Variant Average Binding','Experimental'],
    ['RBD Wuhan-Hu-1 Expression','Experimental'],
    ['RBD Variant Average Expression','Experimental'],

    ['relative_grammaticality','ESM'],
    ['semantic_score','ESM'],
    ['relative_grammaticality_with_eve','ESM'],
    ['grammaticality','ESM'],
    ['masked_grammaticality','ESM'],
    ['mutated_grammaticality','ESM'],

    ['IND Mutability Score','Mutability'],
    ['DCA Mutability Score','Mutability'],

    ['Evescape','Eve'],
    ['Evescape Fitness','Eve'],

    ]

proportions = [0.1,0.25]

In [10]:
monthly = muts_per_month(
        dms,
        datasets,
        mut_counts_per_month,
        proportions
    )
monthly

,date,count,percentage,proportion,feature,method,region
0,2020-03,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-04,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-05,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-06,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-07,0,0.0,0.1,Entry,Experimental Full Spike,spike
...,...,...,...,...,...,...,...
0,2022-04,204,0.197101,0.25,Evescape Fitness,Eve,rbd
0,2022-05,208,0.200966,0.25,Evescape Fitness,Eve,rbd
0,2022-06,214,0.206763,0.25,Evescape Fitness,Eve,rbd
0,2022-07,219,0.211594,0.25,Evescape Fitness,Eve,rbd


In [141]:
monthly.to_csv('monthly_muts_cumulative.csv',index=False)

In [11]:
common_dms = dms.dropna(how='any',subset = ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
    'relative_grammaticality_with_eve',
    'Evescape Fitness',
    'Evescape',
    'IND Mutability Score','DCA Mutability Score',
    'Entry',
    'Binding',
    'Escape',
      # 'RBD Wuhan-Hu-1 Binding',
      # 'RBD Variant Average Binding',
      # 'RBD Wuhan-Hu-1 Expression',
      # 'RBD Variant Average Expression'
             ])
common_dms

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
912,R44K,0.485832,-3.113522,0.016632,0.102939,-2.684610,-3.952643,-430.09720,-0.736182,-3.523731,...,NTD,basic,basic,basic,basic,-2.168124,-0.872255,-2.616702,-2.637246,25.0
925,S45A,0.526944,-3.662286,0.093964,-0.063606,-2.507967,-4.010969,-430.01987,-0.412289,-2.856649,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-1.222299,-2.262846,-2.363303,67.0
929,S45F,0.882909,-3.485551,0.113342,-0.010517,-2.306809,-3.834234,-430.00050,-0.359200,-2.655491,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.902856,-0.559057,-2.132056,-2.220246,724.0
932,S45I,0.955088,-3.652334,-0.255646,-0.078481,-2.564907,-4.001016,-430.36948,-0.427164,-2.913590,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-0.337976,-2.105012,-2.236893,NaN
937,S45P,0.458789,-3.876604,-0.209167,-0.165349,-2.607943,-4.225287,-430.32300,-0.514031,-2.956625,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-1.811848,-2.392182,-2.471186,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26273,S1252C,0.531152,-4.484754,0.108093,-0.081452,-2.660791,-4.616760,-430.00574,-0.213458,-2.792798,...,CT,polar/neutral,hydroxylic,polar/neutral,sulphur-containing,-1.736454,-1.609191,-2.161179,-1.963374,350.0
26276,S1252F,0.471660,-6.508742,-0.653198,-0.929373,-4.246643,-6.640748,-430.76703,-1.061379,-4.378649,...,CT,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.736454,-0.559057,-2.120564,-2.307275,13637.0
26284,S1252P,0.532022,-7.721882,-2.379517,-2.027274,-5.555630,-7.853888,-432.49335,-2.159280,-5.687636,...,CT,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.736454,-1.811848,-2.463053,-2.561109,3693.0
26288,S1252T,0.333992,-5.059584,-0.807220,-0.444661,-2.923320,-5.191590,-430.92105,-0.576667,-3.055326,...,CT,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-1.736454,-2.014505,-2.284323,-2.441923,22.0


In [29]:
common_dms[(common_dms.pos>319) & (common_dms.pos<541)].sort_values('RBD Wuhan-Hu-1 Binding',ascending=False).head(10)

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
10505,N501F,0.590328,-3.561572,-2.041046,-0.005709,-2.467020,-3.943914,-432.15488,-0.388052,-2.849362,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.107654,0.141032,-1.285934,-1.356395,25.0
9497,Y453F,0.608079,-3.567303,-0.118958,-0.070116,-2.588785,-3.939361,-430.23280,-0.442174,-2.960842,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aromatic,-1.335114,-1.074912,-1.616407,-1.664708,2066.0
10520,N501Y,0.540693,-3.505434,-2.235535,0.025003,-2.383195,-3.887776,-432.34937,-0.357339,-2.765537,...,RBD,polar/neutral,hydroxylic,polar/neutral,aromatic,-1.107654,-0.945948,-1.465179,-1.542303,6743345.0
10517,N501T,0.397961,-3.125689,0.328247,0.066789,-2.019247,-3.508031,-429.78558,-0.315553,-2.401589,...,RBD,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-1.107654,-1.314416,-1.510965,-1.516599,7943.0
10008,S477N,0.333799,-3.694978,-0.219543,-0.127404,-2.531080,-4.026671,-430.33337,-0.459097,-2.862773,...,RBD,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-0.777370,-1.461803,-1.472985,-1.554482,5519217.0
10511,N501M,0.638707,-5.199243,-1.723999,-0.980809,-4.139781,-5.581586,-431.83783,-1.363152,-4.522124,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,sulphur-containing,-1.107654,-0.503787,-1.502061,-1.799268,NaN
10009,S477P,0.460109,-4.035852,0.432892,-0.195418,-2.826552,-4.367545,-429.68094,-0.527111,-3.158245,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-0.777370,-1.811848,-1.564637,-1.709606,NaN
10444,Q498H,0.614069,-4.396212,-0.020294,-0.624948,-3.712381,-5.082502,-430.13412,-1.311238,-4.398671,...,RBD,polar/neutral,hydroxylic,basic,basic,-1.059591,0.164440,-1.317085,-1.508689,42.0
9501,Y453K,1.155215,-3.960115,-0.177338,-0.196475,-2.813566,-4.332172,-430.29117,-0.568532,-3.185624,...,RBD,polar/neutral,aromatic,basic,basic,-1.335114,1.693582,-1.226219,-1.381985,NaN
9502,Y453L,0.878154,-3.539882,-0.498444,-0.066144,-2.531579,-3.911939,-430.61227,-0.438201,-2.903637,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aliphatic,-1.335114,-1.222299,-1.642282,-1.649420,NaN


In [ ]:
common_dms[(common_dms.pos>319) & (common_dms.pos<541)].sort_values('Binding',ascending=False).head(10)

In [30]:
common_dms_rbd = common_dms.dropna(how='any',subset = ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
    'relative_grammaticality_with_eve',
    'Evescape Fitness',
    'Evescape',
    'IND Mutability Score','DCA Mutability Score',
    'Entry',
    'Binding',
    'Escape',
      'RBD Wuhan-Hu-1 Binding',
      'RBD Variant Average Binding',
      'RBD Wuhan-Hu-1 Expression',
      'RBD Variant Average Expression'
             ])
common_dms_rbd

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
6933,N331D,0.890939,-5.042127,-1.089111,-0.374183,-2.765436,-5.107607,-431.20294,-0.439664,-2.830916,...,RBD,polar/neutral,hydroxylic,acidic,acidic,-1.366844,-0.222451,-1.589168,-1.553617,5.0
6934,N331E,0.872587,-5.323605,-0.708740,-0.372871,-2.723860,-5.389085,-430.82257,-0.438352,-2.789340,...,RBD,polar/neutral,hydroxylic,acidic,acidic,-1.366844,-0.332992,-1.628173,-1.623147,NaN
6937,N331H,0.978823,-6.952316,-1.878723,-1.481362,-4.618758,-7.017797,-431.99255,-1.546842,-4.684238,...,RBD,polar/neutral,hydroxylic,basic,basic,-1.366844,0.072323,-1.694638,-1.900182,137.0
6945,N331R,1.166395,-5.556024,-2.013000,-0.634928,-3.436483,-5.621505,-432.12683,-0.700409,-3.501964,...,RBD,polar/neutral,hydroxylic,basic,basic,-1.366844,1.767276,-1.352608,-1.480823,NaN
6946,N331S,0.864395,-4.876938,-1.080658,-0.333240,-2.589011,-4.942419,-431.19450,-0.398720,-2.654491,...,RBD,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-1.366844,-1.461803,-1.799483,-1.752839,390.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,A522T,0.311180,-2.721524,0.353729,0.220651,-1.847089,-3.227463,-429.76010,-0.285288,-2.353028,...,RBD,nonpolar/hydrophobic,aliphatic,polar/neutral,hydroxylic,-1.819817,-1.369686,-2.114248,-2.146949,90.0
10959,A522V,0.523341,-2.633367,0.281952,0.161167,-1.905230,-3.139306,-429.83188,-0.344772,-2.411169,...,RBD,nonpolar/hydrophobic,aliphatic,nonpolar/hydrophobic,aliphatic,-1.819817,-1.627614,-2.159792,-2.193610,11519.0
10963,T523A,0.350147,-3.940734,0.461609,-0.169708,-2.600693,-4.217108,-429.65222,-0.446082,-2.877067,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.811630,-1.369686,-2.176902,-2.221005,134.0
10970,T523I,0.468424,-4.055481,-0.202271,-0.229018,-2.799499,-4.331855,-430.31610,-0.505393,-3.075873,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.811630,-0.485363,-2.023876,-2.061753,140.0


In [31]:
common_dms_rbd[(common_dms_rbd.pos>319) & (common_dms_rbd.pos<541)].sort_values('RBD Wuhan-Hu-1 Binding',ascending=False).head(10)

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
10505,N501F,0.590328,-3.561572,-2.041046,-0.005709,-2.467020,-3.943914,-432.15488,-0.388052,-2.849362,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.107654,0.141032,-1.285934,-1.356395,25.0
9497,Y453F,0.608079,-3.567303,-0.118958,-0.070116,-2.588785,-3.939361,-430.23280,-0.442174,-2.960842,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aromatic,-1.335114,-1.074912,-1.616407,-1.664708,2066.0
10520,N501Y,0.540693,-3.505434,-2.235535,0.025003,-2.383195,-3.887776,-432.34937,-0.357339,-2.765537,...,RBD,polar/neutral,hydroxylic,polar/neutral,aromatic,-1.107654,-0.945948,-1.465179,-1.542303,6743345.0
10517,N501T,0.397961,-3.125689,0.328247,0.066789,-2.019247,-3.508031,-429.78558,-0.315553,-2.401589,...,RBD,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-1.107654,-1.314416,-1.510965,-1.516599,7943.0
10008,S477N,0.333799,-3.694978,-0.219543,-0.127404,-2.531080,-4.026671,-430.33337,-0.459097,-2.862773,...,RBD,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-0.777370,-1.461803,-1.472985,-1.554482,5519217.0
10511,N501M,0.638707,-5.199243,-1.723999,-0.980809,-4.139781,-5.581586,-431.83783,-1.363152,-4.522124,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,sulphur-containing,-1.107654,-0.503787,-1.502061,-1.799268,NaN
10009,S477P,0.460109,-4.035852,0.432892,-0.195418,-2.826552,-4.367545,-429.68094,-0.527111,-3.158245,...,RBD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-0.777370,-1.811848,-1.564637,-1.709606,NaN
10444,Q498H,0.614069,-4.396212,-0.020294,-0.624948,-3.712381,-5.082502,-430.13412,-1.311238,-4.398671,...,RBD,polar/neutral,hydroxylic,basic,basic,-1.059591,0.164440,-1.317085,-1.508689,42.0
9501,Y453K,1.155215,-3.960115,-0.177338,-0.196475,-2.813566,-4.332172,-430.29117,-0.568532,-3.185624,...,RBD,polar/neutral,aromatic,basic,basic,-1.335114,1.693582,-1.226219,-1.381985,NaN
9502,Y453L,0.878154,-3.539882,-0.498444,-0.066144,-2.531579,-3.911939,-430.61227,-0.438201,-2.903637,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aliphatic,-1.335114,-1.222299,-1.642282,-1.649420,NaN


In [32]:
common_dms_rbd[(common_dms_rbd.pos>319) & (common_dms_rbd.pos<541)].sort_values('Binding',ascending=False).head(10)

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
10245,C488V,1.380514,-1.176009,0.430573,1.326919,-1.110108,-2.920954,-429.68326,-0.418026,-2.855053,...,RBD,polar/neutral,sulphur-containing,nonpolar/hydrophobic,aliphatic,-1.029811,-1.240723,-1.362961,-1.358435,NaN
8067,T385D,0.648506,-3.959823,-0.398163,-0.216604,-2.673344,-4.222918,-430.51200,-0.479699,-2.936438,...,RBD,polar/neutral,hydroxylic,acidic,acidic,-1.592304,0.625025,-1.580745,-1.665461,NaN
10765,L513P,0.977178,-3.818541,-5.446045,-0.154634,-2.600874,-4.226212,-435.55988,-0.562305,-3.008545,...,RBD,nonpolar/hydrophobic,aliphatic,nonpolar/hydrophobic,aliphatic,-1.838053,-1.056489,-2.146786,-1.992262,24.0
8497,D405P,0.559521,-4.221617,-0.446106,-0.317226,-3.024080,-4.659573,-430.55994,-0.755182,-3.462036,...,RBD,acidic,acidic,nonpolar/hydrophobic,aliphatic,-1.208273,0.827682,-1.276699,-1.490555,NaN
8605,I410S,0.552243,-4.129679,-1.463654,-0.317338,-2.814450,-4.376552,-431.57748,-0.564211,-3.061323,...,RBD,nonpolar/hydrophobic,aliphatic,polar/neutral,hydroxylic,-1.671085,-0.337976,-1.826293,-2.046124,4.0
8727,G416N,0.399833,-3.583690,-0.168732,-0.088946,-2.897579,-4.265438,-430.28256,-0.770694,-3.579327,...,RBD,polar/neutral,aliphatic,polar/neutral,hydroxylic,-1.298896,-0.688021,-1.524119,-1.629990,NaN
8454,R403N,0.629572,-2.349387,0.224182,0.363769,-1.576563,-2.945991,-429.88965,-0.232836,-2.173168,...,RBD,basic,basic,polar/neutral,hydroxylic,-1.394068,1.767276,-1.163767,-1.144773,NaN
7407,W353R,0.946696,-2.299789,-0.117310,0.643955,-2.082060,-3.660039,-430.23114,-0.716295,-3.442309,...,RBD,nonpolar/hydrophobic,aromatic,basic,basic,-1.600376,3.628039,-1.171714,-1.156906,18.0
8828,Y421I,0.494065,-3.171737,0.204437,0.003008,-2.150721,-3.562800,-429.90940,-0.388055,-2.541784,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aliphatic,-1.234265,-0.853831,-1.490277,-1.487475,NaN
9497,Y453F,0.608079,-3.567303,-0.118958,-0.070116,-2.588785,-3.939361,-430.23280,-0.442174,-2.960842,...,RBD,polar/neutral,aromatic,nonpolar/hydrophobic,aromatic,-1.335114,-1.074912,-1.616407,-1.664708,2066.0


In [249]:
common_monthly_rbd = muts_per_month(
        common_dms,
        datasets,
        mut_counts_per_month,
        proportions
    )
common_monthly_rbd

,date,count,percentage,proportion,feature,method,region
0,2020-03,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-04,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-05,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-06,0,0.0,0.1,Entry,Experimental Full Spike,spike
0,2020-07,0,0.0,0.1,Entry,Experimental Full Spike,spike
...,...,...,...,...,...,...,...
0,2022-04,115,0.410714,0.25,Evescape Fitness,Eve,rbd
0,2022-05,116,0.414286,0.25,Evescape Fitness,Eve,rbd
0,2022-06,120,0.428571,0.25,Evescape Fitness,Eve,rbd
0,2022-07,121,0.432143,0.25,Evescape Fitness,Eve,rbd


In [250]:
common_monthly_rbd.to_csv('common_monthly_muts_cumulative.csv',index=False)

# VOC Mutations in all measurments

In [135]:
variant_counts(common_dms,mutations).to_csv('voc_indentification_updated_common_measurements.csv')

# Split on RBD and not RBD

In [252]:
common_dms_spike = dms.dropna(how='any',subset = ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
    'relative_grammaticality_with_eve',
    'Evescape Fitness',
    'Evescape',
    'IND Mutability Score','DCA Mutability Score',
    'Entry',
    'Binding',
    'Escape'
    ])
common_dms_spike

,label,semantic_score,relative_grammaticality,relative_sequence_grammaticality,relative_mutated_grammaticality,relative_masked_grammaticality,grammaticality,sequence_grammaticality,mutated_grammaticality,masked_grammaticality,...,domain,reference_amino_acid_property_basic,reference_amino_acid_property_complex,mutant_amino_acid_property_basic,mutant_amino_acid_property_complex,Accessibility WCN,Dissimilarity Charge Hydro,relative_grammaticality_with_eve,ESM-1b Relative Grammaticality with EVE,counts
912,R44K,0.485832,-3.113522,0.016632,0.102939,-2.684610,-3.952643,-430.09720,-0.736182,-3.523731,...,NTD,basic,basic,basic,basic,-2.168124,-0.872255,-2.616702,-2.637246,25.0
925,S45A,0.526944,-3.662286,0.093964,-0.063606,-2.507967,-4.010969,-430.01987,-0.412289,-2.856649,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-1.222299,-2.262846,-2.363303,67.0
929,S45F,0.882909,-3.485551,0.113342,-0.010517,-2.306809,-3.834234,-430.00050,-0.359200,-2.655491,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.902856,-0.559057,-2.132056,-2.220246,724.0
932,S45I,0.955088,-3.652334,-0.255646,-0.078481,-2.564907,-4.001016,-430.36948,-0.427164,-2.913590,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-0.337976,-2.105012,-2.236893,NaN
937,S45P,0.458789,-3.876604,-0.209167,-0.165349,-2.607943,-4.225287,-430.32300,-0.514031,-2.956625,...,NTD,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.902856,-1.811848,-2.392182,-2.471186,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26273,S1252C,0.531152,-4.484754,0.108093,-0.081452,-2.660791,-4.616760,-430.00574,-0.213458,-2.792798,...,CT,polar/neutral,hydroxylic,polar/neutral,sulphur-containing,-1.736454,-1.609191,-2.161179,-1.963374,350.0
26276,S1252F,0.471660,-6.508742,-0.653198,-0.929373,-4.246643,-6.640748,-430.76703,-1.061379,-4.378649,...,CT,polar/neutral,hydroxylic,nonpolar/hydrophobic,aromatic,-1.736454,-0.559057,-2.120564,-2.307275,13637.0
26284,S1252P,0.532022,-7.721882,-2.379517,-2.027274,-5.555630,-7.853888,-432.49335,-2.159280,-5.687636,...,CT,polar/neutral,hydroxylic,nonpolar/hydrophobic,aliphatic,-1.736454,-1.811848,-2.463053,-2.561109,3693.0
26288,S1252T,0.333992,-5.059584,-0.807220,-0.444661,-2.923320,-5.191590,-430.92105,-0.576667,-3.055326,...,CT,polar/neutral,hydroxylic,polar/neutral,hydroxylic,-1.736454,-2.014505,-2.284323,-2.441923,22.0


In [27]:
def muts_per_month_rbd_and_outwith_rbd(scores, feature_methods, monthly_counts, proportions):
    all_values = []
    for region in ['spike','rbd']:
        if region == 'rbd':
            region_scores = scores.dropna(how='any',subset = 
                                           ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
                                            'relative_grammaticality_with_eve',
                                            'Evescape Fitness',
                                            'Evescape',
                                            'IND Mutability Score','DCA Mutability Score',
                                            'Entry',
                                            'Binding',
                                            'Escape',
                                            'RBD Wuhan-Hu-1 Binding',
                                            'RBD Variant Average Binding',
                                            'RBD Wuhan-Hu-1 Expression',
                                            'RBD Variant Average Expression'
                                            ])
            region_monthly = monthly_counts[monthly_counts.RBD == True]
        else:
            region_scores =  scores.dropna(how='any',subset = 
                                           ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
                                            'relative_grammaticality_with_eve',
                                            'Evescape Fitness',
                                            'Evescape',
                                            'IND Mutability Score','DCA Mutability Score',
                                            'Entry',
                                            'Binding',
                                            'Escape'
                                            ])
            region_monthly = monthly_counts
        for proportion in proportions:
            for value in feature_methods:
                #Top Proportion
                sorted_scores = region_scores.sort_values(value[0],ascending=False)
                sorted_scores = sorted_scores.dropna(subset=value[0])                
                subset_length = len(sorted_scores)
                print(subset_length,value,region)
                sorted_scores = sorted_scores[:int(subset_length*proportion)]

                #Subset in data
                monthly_mutations =region_monthly.merge(sorted_scores,left_on='mutation',right_on='label',how='left') 
                monthly_mutations=monthly_mutations[monthly_mutations.label.notnull()]
                for date in monthly_counts.sample_date.unique():
                    if subset_length == 0 | (region == 'rbd'  and 'RBD' in value[0]):
                        row = pd.DataFrame([date,0,0,proportion,value[0],value[1],region],index = ['date','count','percentage','proportion','feature','method','region']).T
                        all_values.append(row)
                        continue
                    else:
                        subset = monthly_mutations[monthly_mutations.sample_date <= date]
                        row = pd.DataFrame([date,len(subset),len(subset)/len(sorted_scores),proportion,value[0],value[1],region],index = ['date','count','percentage','proportion','feature','method','region']).T
                        all_values.append(row)
    return pd.concat(all_values)

In [28]:
common = muts_per_month_rbd_and_outwith_rbd(
        dms,
        datasets,
        mut_counts_per_month,
        [0.1]
    )
common.to_csv('common_monthly_muts_cumulative_all.csv',index=False)

4662 ['Entry', 'Experimental Full Spike'] spike
4662 ['Binding', 'Experimental Full Spike'] spike
4662 ['Escape', 'Experimental Full Spike'] spike
1048 ['RBD Wuhan-Hu-1 Binding', 'Experimental'] spike
1049 ['RBD Variant Average Binding', 'Experimental'] spike
1036 ['RBD Wuhan-Hu-1 Expression', 'Experimental'] spike
1049 ['RBD Variant Average Expression', 'Experimental'] spike
4662 ['relative_grammaticality', 'ESM'] spike
4662 ['semantic_score', 'ESM'] spike
4662 ['relative_grammaticality_with_eve', 'ESM'] spike
4662 ['grammaticality', 'ESM'] spike
4662 ['masked_grammaticality', 'ESM'] spike
4662 ['mutated_grammaticality', 'ESM'] spike
4662 ['IND Mutability Score', 'Mutability'] spike
4662 ['DCA Mutability Score', 'Mutability'] spike
4662 ['Evescape', 'Eve'] spike
4662 ['Evescape Fitness', 'Eve'] spike
1036 ['Entry', 'Experimental Full Spike'] rbd
1036 ['Binding', 'Experimental Full Spike'] rbd
1036 ['Escape', 'Experimental Full Spike'] rbd
1036 ['RBD Wuhan-Hu-1 Binding', 'Experimental'

In [24]:
common[(common.region == 'spike') & (common.feature == 'RBD Variant Average Binding')]['count'].sum()

0

In [25]:
def variant_counts_rbd_and_outwith(scores,voc_mutations):

  score_counts = []
  proportions = [[0,0.1],[0.1,0.25],[0.25,1]]

  common_scores_spike = scores.dropna(how='any',subset = 
                                           ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
                                            'relative_grammaticality_with_eve',
                                            'Evescape Fitness',
                                            'Evescape',
                                            'IND Mutability Score','DCA Mutability Score',
                                            'Entry',
                                            'Binding',
                                            'Escape'
                                            ])
  common_scores_rbd = common_scores_spike.dropna(how='any',subset = 
                                           ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",
                                            'relative_grammaticality_with_eve',
                                            'Evescape Fitness',
                                            'Evescape',
                                            'IND Mutability Score','DCA Mutability Score',
                                            'Entry',
                                            'Binding',
                                            'Escape',
                                            'RBD Wuhan-Hu-1 Binding',
                                            'RBD Variant Average Binding',
                                            'RBD Wuhan-Hu-1 Expression',
                                            'RBD Variant Average Expression'
                                            ])
  
  for proportion in proportions:
    for score in ['semantic_score','relative_grammaticality',"masked_grammaticality","mutated_grammaticality",'relative_grammaticality_with_eve',
                  'Evescape Fitness','Evescape','IND Mutability Score','DCA Mutability Score','Entry','Binding','Escape','RBD Wuhan-Hu-1 Binding',
                  'RBD Variant Average Binding','RBD Wuhan-Hu-1 Expression','RBD Variant Average Expression']:
        
        
        #Sort DMS on score
        sorted_dms = common_scores_spike.sort_values(score,ascending=False)
        # sorted_dms = sorted_dms[(sorted_dms.pos < 319) | (sorted_dms.pos > 541)]
        subset_length = len(sorted_dms.dropna(subset=score))
        sorted_dms = sorted_dms[int(subset_length*proportion[0]):int(subset_length*proportion[1])]

        #Full Spike Row
        full_spike = pd.merge(sorted_dms,voc_mutations,left_on='label',right_on='mutation',how='left')[["B.1.1.7","B.1.351","B.1.617.2","P.1","BA.1","BA.2","BA.4","BA.2.12.1","BA.2.75"]]
        row = pd.DataFrame(full_spike.sum()).T
        if 'RBD' in score:
            row['feature'] = 0
        else:
            row['feature'] = score
        row['region'] = 'spike'
        row['proportion'] = proportion[1]
        score_counts.append(row)

        

        
        #Sort DMS on score
        sorted_rbd_dms = common_scores_rbd.sort_values(score,ascending=False)
        sorted_rbd_dms = sorted_rbd_dms[(sorted_rbd_dms.pos >= 319) & (sorted_rbd_dms.pos <= 541)]
        rbd_subset_length = len(sorted_rbd_dms.dropna(subset=score))
        sorted_rbd_dms = sorted_rbd_dms[int(rbd_subset_length*proportion[0]):int(rbd_subset_length*proportion[1])]
        #RBD Row
        rbd = pd.merge(sorted_rbd_dms,voc_mutations,left_on='label',right_on='mutation',how='left')[["B.1.1.7","B.1.351","B.1.617.2","P.1","BA.1","BA.2","BA.4","BA.2.12.1","BA.2.75"]]
        row = pd.DataFrame(rbd.sum()).T
        row['feature'] = score
        row['region'] = 'rbd'
        row['proportion'] = proportion[1]
        score_counts.append(row)
  score_counts = pd.concat(score_counts) 
  score_counts = score_counts.melt(id_vars=['feature','region','proportion'],var_name='voc',value_name='count')
  return score_counts

In [26]:
variant_counts_rbd_and_outwith(scores = dms,voc_mutations=mutations).to_csv('voc_indentification_updated_common_measurements_rbd_spike.csv',index=False)